In [1]:
import pandas as pd
import numpy as np
import sklearn
import panel as pn
import re
from bokeh.plotting import figure, show
from bokeh.models import BasicTicker, ColorBar, LinearColorMapper, PrintfTickFormatter
pn.extension(comms='vscode')

In [2]:
assets_df = pd.read_csv('data/jordans_asset_dataset_2023_03_06.csv')
assets_full_df = assets_df.copy()
assets_df.head()

,asset.cost_center,asset.u_asset_id,assigned_to,assigned_to.active,assigned_to.cost_center,assigned_to.department,assigned_to.employee_number,assigned_to.manager,BioAccess.bioFire Email Address,assigned_to.u_company_code,...,Sophos.OS,Sophos.Protection,Sophos.Tamper protection,support_group,u_ci,u_status,u_sub_status,warranty_expiration,sys_created_on,sys_created_on.1
0,2860747,AST0108027,MECHAM Jonathan,True,2860747,Software Development,16012062,MCCARTY Jameson,jonathan.mecham@biofiredx.com,028 - BioFire Diagnostics LLC,...,Windows 10 Enterprise,Intercept X Advanced with XDR,On,BMX-Deskside_SLC_L2,True,Deployed,Functional,2024-12-03 00:00:00,2022-12-09 00:00:00,2022-12-09 00:00:00
1,2858030,AST0109801,ALLEN Jared,True,2858030,BFDX IS Internal Business Software,10019166,DAVISON Kirby,Jared.Allen@biofiredx.com,028 - BioFire Diagnostics LLC,...,Windows 10 Enterprise,Intercept X Advanced with XDR,On,BMX-Deskside_SLC_L2,True,Deployed,Functional,2023-11-06 00:00:00,2022-12-09 00:00:00,2022-12-09 00:00:00
2,2860705,AST0115321,BHATIA Ajay,True,2860705,Regulatory Affairs,10018360,BOURZAC Kevin,Ajay.Bhatia@biofiredx.com,028 - BioFire Diagnostics LLC,...,Windows 10 Enterprise,Intercept X Advanced with XDR,On,BMX-Deskside_SLC_L2,True,Deployed,Functional,2025-08-16 00:00:00,2022-12-21 00:00:00,2022-12-21 00:00:00
3,2121010,AST0115322,CLARKE Kimon,True,2121010,US CLNMKT Hospital-Based Clinician,10022276,SHAFFER Matt,Kimon.Clarke@biofiredx.com,021 - bioMérieux Inc.,...,Windows 10 Enterprise,Intercept X Advanced with XDR,Off,BMX-Deskside_SLC_L2,True,Deployed,Functional,2025-08-17 00:00:00,2022-12-22 00:00:00,2022-12-22 00:00:00
4,2804740,AST0115323,SMITH Jennifer,True,2804740,Manufacturing QA Measurement Systems,10021946,BALMANNO Sam,Jennifer.Smith@biofiredx.com,028 - BioFire Diagnostics LLC,...,Windows 10 Enterprise,Intercept X Advanced with XDR,On,BMX-Deskside_SLC_L2,True,Deployed,Functional,2025-08-17 00:00:00,2022-12-22 00:00:00,2022-12-22 00:00:00


In [3]:
for col in assets_df.columns:
    print (col)

asset.cost_center
asset.u_asset_id
assigned_to
assigned_to.active
assigned_to.cost_center
assigned_to.department
assigned_to.employee_number
assigned_to.manager
BioAccess.bioFire Email Address
assigned_to.u_company_code
BioAccess.Display Name
BioAccess.Email
BioAccess.Employee ID
BioAccess.Employee Status
BioAccess.Username
category
Computer Name
BioAccess.Sunset Date
Intune.Azure AD Device ID
Intune.Azure AD registered
Intune.Category
Intune.CellularTechnology
Intune.Compliance
Intune.Computer Name
Intune.Device ID
Intune.Device state
Intune.EAS activated
Intune.EAS activation ID
Intune.EAS reason
Intune.EAS status
Intune.EID
Intune.Encrypted
Intune.Enrollment date
Intune.EthernetMAC
Intune.Free storage
Intune.ICCID
Intune.IMEI
Intune.Intune registered
Intune.Jailbroken
Intune.JoinType
Intune.Last check-in
Intune.Managed by
Intune.Management name
Intune.Manufacturer
Intune.MEID
Intune.Model
Intune.OS
Intune.OS version
Intune.Ownership
Intune.Phone number
Intune.Primary user display na

In [4]:
computers_by_employee_id_df = assets_df.loc[:,['assigned_to.employee_number', 'Computer Name']].groupby(['assigned_to.employee_number']).count()
computers_by_employee_id_df.columns = ["user_asset_count"]
# computers_by_employee_id_df.loc[computers_by_employee_id_df.loc[:,"user_asset_count"] == 0 ]
assets_df = assets_df.merge(computers_by_employee_id_df, on="assigned_to.employee_number")

In [7]:
assets_df.groupby(['assigned_to.department']).count()

,asset.cost_center,asset.u_asset_id,assigned_to,assigned_to.active,assigned_to.cost_center,assigned_to.employee_number,assigned_to.manager,BioAccess.bioFire Email Address,assigned_to.u_company_code,BioAccess.Display Name,...,Sophos.Protection,Sophos.Tamper protection,support_group,u_ci,u_status,u_sub_status,warranty_expiration,sys_created_on,sys_created_on.1,user_asset_count
assigned_to.department,,,,,,,,,,,,,,,,,,,,,
2103103,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
Accounting Services US,21,21,21,21,21,21,21,21,21,21,...,21,21,21,21,21,21,21,21,21,21
Accounting US,9,9,9,9,9,9,9,9,9,9,...,9,9,9,9,9,9,9,9,9,9
Americas HR Operations,5,5,5,5,5,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
Americas manufacturing controlling,5,5,5,5,5,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
US Hardware,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
US Marketing,7,7,7,7,7,7,7,7,7,7,...,7,7,7,7,7,7,7,7,7,7
US Marketing Communication,14,14,14,14,14,14,14,14,14,14,...,14,14,14,14,14,14,14,14,14,14


In [20]:
missing_value_count_df = len(assets_df) - assets_df.count()
missing_value_count_df.loc[missing_value_count_df > 0]


BioAccess.Employee ID       17
BioAccess.Username          17
BioAccess.Sunset Date       17
Intune.Enrollment date       7
Intune.Last check-in         7
Intune.OS version            7
ISAOS.Description         2685
location                     1
MDE.Device IPs               1
dtype: int64

In [24]:
assets_df["ISAOS.Description"].loc[assets_df.loc[:,"ISAOS.Description"].notna()]

18                                    Microspotter System
192                                                BioMek
199                              Used for Waters Software
212                              Used for Waters Software
250                                   Microspotter System
252                                   Microspotter System
253                                   Microspotter System
256                                   Microspotter System
261                                   Microspotter System
780                                   Craig's Test Laptop
878                                             BioChem 2
879                                        Receiving Dock
880          Raw Materials Warehouse(Receiving Warehouse)
881             Array Automation 1 (Laura Hutchings) left
882            Array Automation 2 (Laura Hutchings) right
883       515 Array by gowning area - outside Office 1203
884                             515 Array by manual lines
885           

In [26]:
assets_df.drop("ISAOS.Description", axis="columns", inplace=True)

In [29]:
assets_df[assets_df['Intune.Free storage'].eq("None Listed")].count()

asset.cost_center          125
asset.u_asset_id           125
assigned_to                125
assigned_to.active         125
assigned_to.cost_center    125
                          ... 
u_sub_status               125
warranty_expiration        125
sys_created_on             125
sys_created_on.1           125
user_asset_count           125
Length: 126, dtype: int64

In [32]:
assets_df[assets_df['Intune.Jailbroken'].eq("None Listed")].count()

asset.cost_center          125
asset.u_asset_id           125
assigned_to                125
assigned_to.active         125
assigned_to.cost_center    125
                          ... 
u_sub_status               125
warranty_expiration        125
sys_created_on             125
sys_created_on.1           125
user_asset_count           125
Length: 126, dtype: int64

In [49]:
def search_in_features(search_pattern:str, case_sensitive = False) -> list[str]:
    search_list = []
    for col_name in assets_df.columns:
        search_area = col_name.split('.').pop(-1)
        if case_sensitive:
            result = re.search(search_pattern, search_area, re.IGNORECASE)
        else:
            result = re.search(search_pattern, search_area)
        if result:
            print(col_name)
            search_list.append(col_name)
    return search_list


In [51]:
assets_df.loc[:,search_in_features("OS", True) + search_in_features('Operating System')]


asset.cost_center
assigned_to.cost_center
Intune.OS
Intune.OS version
Intune.SystemManagementBIOSVersion
MDE.Exposure Level
MDE.OS Build
MDE.OS Distribution
MDE.OS Platform
MDE.OS Version
Sophos.OS
ISAOS.Operating System Version Number
ISAOS.Operating System


,asset.cost_center,assigned_to.cost_center,Intune.OS,Intune.OS version,Intune.SystemManagementBIOSVersion,MDE.Exposure Level,MDE.OS Build,MDE.OS Distribution,MDE.OS Platform,MDE.OS Version,Sophos.OS,ISAOS.Operating System Version Number,ISAOS.Operating System
0,2860747,2860747,Windows,10.0.19044.2486,1.5.1,None Listed,0,None Listed,None Listed,None Listed,Windows 10 Enterprise,10.0 (19044),Windows 10 Enterprise
1,2858030,2858030,Windows,10.0.19044.2486,1.9.0,No data available,19042,Windows10,Windows10,20H2,Windows 10 Enterprise,10.0 (19044),Windows 10 Enterprise
2,2860705,2860705,Windows,10.0.19044.2364,1.10.0,None Listed,0,None Listed,None Listed,None Listed,Windows 10 Enterprise,10.0 (19044),Windows 10 Pro
3,2860705,2860705,Windows,10.0.19042.2604,1.24.0,Medium,19042,Windows10,Windows10,20H2,Windows 10 Enterprise,10.0 (19042),Windows 10 Enterprise
4,2121010,2121010,Windows,10.0.19044.2604,1.9.3,Low,19044,Windows10,Windows10,21H2,Windows 10 Enterprise,10.0 (19044),Windows 10 Enterprise
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2702,2858020,2858020,Windows,10.0.19044.2604,1.9.1,Medium,19044,Windows10,Windows10,21H2,Windows 10 Enterprise,10.0 (19044),Windows 10 Enterprise
2703,2850100,2850100,Windows,10.0.19044.2486,1.9.1,Medium,19044,Windows10,Windows10,21H2,Windows 10 Enterprise,10.0 (19044),Windows 10 Enterprise
2704,2860740,2860740,Windows,10.0.19044.2486,1.8.0,Medium,19044,Windows10,Windows10,21H2,Windows 10 Enterprise,10.0 (19044),Windows 10 Enterprise
2705,2858100,2858100,Windows,10.0.19044.2486,1.9.1,None Listed,0,None Listed,None Listed,None Listed,Windows 10 Enterprise,10.0 (19044),Windows 10 Enterprise
